<a href="https://colab.research.google.com/github/pmj-chosim/Commit-Project-2023.1.20-2023.2.28-/blob/main/2023.02.16/gpt-neo%20finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.21.4
!pip install pandas==1.2.2
!pip install torch
!pip install torchvision
!pip install torchsummary==1.5.1
!pip install transformers==4.26.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.2.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer
from tokenizers import Tokenizer

torch.manual_seed(42)

tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
bos_token = '<|startoftext|>'
eos_token = '<|endoftext|>'
pad_token = '<|pad|>'
tokenizer.add_special_tokens({
    'pad_token': pad_token,
    'bos_token': bos_token,
    'eos_token': eos_token
})
bos_token_id = tokenizer.convert_tokens_to_ids(bos_token)
eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_id = tokenizer.convert_tokens_to_ids(pad_token)
print(tokenizer)
print(bos_token_id,eos_token_id,pad_token_id)

GPT2Tokenizer(name_or_path='EleutherAI/gpt-neo-125M', vocab_size=50257, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|pad|>'})
50258 50256 50257


In [3]:
from transformers import GPTNeoForCausalLM

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()
model.resize_token_embeddings(len(tokenizer))


Embedding(50259, 768)

In [11]:
from google.colab import files

In [12]:
descriptions = pd.read_csv('en_strings.csv')['description']
max_length = max([len(tokenizer.encode(description)) for description in descriptions])
print("Max length: {}".format(max_length))

Max length: 98


In [13]:
class CustomDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


dataset = CustomDataset(descriptions, tokenizer, max_length=max_length)

train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

print(len(train_dataset),len(val_dataset))

900 100


In [14]:
from transformers import TrainingArguments, Trainer, IntervalStrategy

training_args = TrainingArguments(output_dir='./results', num_train_epochs=10, logging_steps=100,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=16, per_device_eval_batch_size=16,
                                  warmup_steps=100, weight_decay=0.01, learning_rate=1e-5, logging_dir='./logs')
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 900
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 570
  Number of trainable parameters = 125200128


Step,Training Loss
100,3.467400
200,1.259300
300,1.166900
400,1.141600
500,1.105000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=570, training_loss=1.5619772392406799, metrics={'train_runtime': 266.4398, 'train_samples_per_second': 33.779, 'train_steps_per_second': 2.139, 'total_flos': 449970038784000.0, 'train_loss': 1.5619772392406799, 'epoch': 10.0})

In [15]:
generated = tokenizer("A medical group has ", return_tensors="pt").input_ids.cuda()
print(generated)
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                bos_token_id=bos_token_id,
                                eos_token_id=eos_token_id, pad_token_id=pad_token_id,
                                max_length=100, top_p=0.95, temperature=0.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.0"
}



tensor([[  32, 3315, 1448,  468,  220]], device='cuda:0')
0: A medical group has  been establishing a "cocaine epidemic" in the United States.
1: A medical group has  been criticized for repeatedly criticizing government medical centers for failing to provide services to patients during the war.
2: A medical group has  been using an automated system for the diagnosis and treatment of multiple conditions in the elderly.
3: A medical group has  supported the establishment of a standard framework for clinical research, and it has also established several basic concepts that can be applied to
4: A medical group has  said that the level of cooperation that was needed among members of the medical group in Brazil is low.
5: A medical group has  been using the term 'pancrathy',
6: A medical group has  been asked to determine whether the United States  has an official medical exemption.
7: A medical group has  stated that the risk of COVID-19 is so great that the level of exposure to COVID-19 i

In [16]:
prompt = (
    "The medical group is "
    #'Where will the korean people\'s assosication of Toronto hell? '
)

In [17]:
test_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()


In [18]:
test_gen_tokens = model.generate(
    test_input_ids,
    do_sample=True, top_k=50,
    bos_token_id=bos_token_id,
    eos_token_id=eos_token_id, 
    pad_token_id=pad_token_id,
    max_length=100, 
    top_p=0.95, 
    temperature=0.1, 
    repetition_penalty=1.5,
    no_repeat_ngram_size=3,
    num_return_sequences=1
)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.0"
}



In [19]:
test_gen_tokens = tokenizer.batch_decode(test_gen_tokens)[0]
print(test_gen_tokens)

The medical group is ifying the need for a more comprehensive and efficient treatment system, which includes a wide range of drugs.<|endoftext|>
